In [ ]:
import cv2
import numpy as np

def calculate_yellow_area(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert to HSV color space (better for color detection)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Define yellow color range in HSV
    # You might need to adjust these values depending on your specific yellow
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([40, 255, 255])
    
    # Create a mask for yellow pixels
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Count non-zero pixels (yellow pixels)
    yellow_pixels = cv2.countNonZero(yellow_mask)
    
    return yellow_pixels

def compare_areas(image1_path, image2_path, tolerance=0.01):
    # Calculate areas
    area1 = calculate_yellow_area(image1_path)
    area2 = calculate_yellow_area(image2_path)

    # Calculate difference percentage
    difference = abs(area1 - area2) / max(area1, area2)

    # Compare with tolerance
    are_equal = difference <= tolerance

    return {
        'are_equal': are_equal,
        'area1': area1,
        'area2': area2,
        'difference_percentage': difference * 100
    }

# Usage example
image1_path = "images/two_shapes2/train/circle/circle_11000_209_314_89_v2.png"
image2_path = "images/two_shapes2/train/star/star_11000_172_287_109_v2.png"

result = compare_areas(image1_path, image2_path)
print(f"Areas are {'equal' if result['are_equal'] else 'different'}")
print(f"Area 1: {result['area1']} pixels")
print(f"Area 2: {result['area2']} pixels")
print(f"Difference: {result['difference_percentage']:.2f}%")

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import re


def calculate_yellow_area(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load image: {image_path}")

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([40, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    return cv2.countNonZero(yellow_mask)


def extract_surface_value(filename):
    # Extract the number between underscores
    match = re.search(r"_(\d+)_", filename)
    if match:
        return int(match.group(1))
    return None


def analyze_images(circle_dir, star_dir):
    # Dictionaries to store areas for each target surface
    circle_areas = defaultdict(list)
    star_areas = defaultdict(list)

    # Process circle images
    print("Processing circle images...")
    for filename in os.listdir(circle_dir):
        if filename.endswith(".png"):
            try:
                surface_value = extract_surface_value(filename)
                if surface_value:
                    area = calculate_yellow_area(os.path.join(circle_dir, filename))
                    circle_areas[surface_value].append(area)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

    # Process star images
    print("Processing star images...")
    for filename in os.listdir(star_dir):
        if filename.endswith(".png"):
            try:
                surface_value = extract_surface_value(filename)
                if surface_value:
                    area = calculate_yellow_area(os.path.join(star_dir, filename))
                    star_areas[surface_value].append(area)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

    # Create summary statistics
    results = []
    all_surfaces = sorted(set(circle_areas.keys()) | set(star_areas.keys()))

    for surface in all_surfaces:
        circle_stats = {
            "mean": np.mean(circle_areas[surface]) if circle_areas[surface] else np.nan,
            "std": np.std(circle_areas[surface]) if circle_areas[surface] else np.nan,
        }
        star_stats = {
            "mean": np.mean(star_areas[surface]) if star_areas[surface] else np.nan,
            "std": np.std(star_areas[surface]) if star_areas[surface] else np.nan,
        }

        results.append(
            {
                "target_surface": surface,
                "circle_mean": circle_stats["mean"],
                "circle_std": circle_stats["std"],
                "star_mean": star_stats["mean"],
                "star_std": star_stats["std"],
                "n_circle_images": len(circle_areas[surface]),
                "n_star_images": len(star_areas[surface]),
            }
        )

    # Create DataFrame
    df = pd.DataFrame(results)
    df = df.sort_values("target_surface")

    return df


# Usage
dir_ = "images/two_shapes3"
circle_dir = f"{dir_}/train/circle" 
star_dir = f"{dir_}/train/star"

try:
    results_df = analyze_images(circle_dir, star_dir)
    # Save to CSV
    results_df.to_csv("shape_analysis_results.csv", index=False)
    print("\nResults have been saved to 'shape_analysis_results.csv'")

except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
# compute the ratios of circle and star to target areas
results_df["circle_ratio"] = results_df["circle_mean"] / results_df["target_surface"]
results_df["star_ratio"] = results_df["star_mean"] / results_df["target_surface"]
# draw a boxplot of the ratios
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.boxplot([results_df["circle_ratio"], results_df["star_ratio"]], labels=["Circle", "Star"])
plt.ylabel("Ratio to Target Surface")   
plt.title("Ratio of Yellow Area to Target Surface")
plt.grid(axis="y")
plt.show()

In [ ]:
results_df

In [ ]:
import numpy as np


def radius_from_area(area: float) -> float:
    """
    Calculate the outer radius of a pentagram given its area.

    Args:
        area: The desired area of the pentagram

    Returns:
        The outer radius needed to create a pentagram with the given area
    """
    # Calculate the geometric constant used in area formula
    inner_to_outer_ratio = np.sin(np.pi / 10) / np.sin(7 * np.pi / 10)
    k = (5 / 2) * np.sin(2 * np.pi / 5) * (1 + inner_to_outer_ratio**2)

    # Return radius
    return np.sqrt(area / k)


def verify_area(radius: float) -> float:
    """
    Verify the calculation by computing the area from the radius.

    Args:
        radius: The outer radius of the pentagram

    Returns:
        The actual area of the pentagram
    """
    inner_radius = radius * np.sin(np.pi / 10) / np.sin(7 * np.pi / 10)
    area = (5 / 2) * np.sin(2 * np.pi / 5) * (radius**2 + inner_radius**2)
    return area


# Example usage
if __name__ == "__main__":
    target_area = 100
    radius = radius_from_area(target_area)
    actual_area = verify_area(radius)

    print(f"Target area: {target_area}")
    print(f"Calculated radius: {radius:.4f}")
    print(f"Verification - actual area: {actual_area:.4f}")